### Choice of robustness metrics:
| Robustness metric 	 | value 	 |
|:-------------------:|:-------:|
|   test          	   | 0    	  |
|          	          |    	    |
|          	          |    	    |

In [21]:
#   import packages
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pickle
import functools

from ema_workbench import (
    Model,
    Policy,
    Scenario,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
    util,
    ScalarOutcome,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.em_framework.optimization import ArchiveLogger, EpsilonProgress
from ema_workbench.em_framework import parameters_from_csv

In [22]:
dike_model, planning_steps = get_model_for_problem_formulation(5)
ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

In [35]:
results = pd.read_pickle(r'generated_datasets\open_exploration_base_policy_worst_scenarios.pkl')

In [36]:
results

,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.5_Expected Annual Damage,A.5_Dike Investment Costs,RfR Total Costs,Expected Evacuation Costs,A.1_Deaths_Cost,A.2_Deaths_Cost,A.3_Deaths_Cost,A.4_Deaths_Cost,A.5_Deaths_Cost,total_cost[€]
2876,75,52.635894,10.0,0.233370,50.689544,1.0,0.028522,103.353196,1.5,0.050318,...,0.000000e+00,0,0.0,0.0,4.724317e+06,1.971384e+06,7.971065e+06,0.000000,0.000000e+00,6.665195e+09
8630,89,33.994820,1.5,0.116091,107.433393,10.0,0.031524,119.060586,10.0,0.006063,...,0.000000e+00,0,0.0,0.0,4.860992e+06,2.056688e+06,7.809525e+06,0.000000,0.000000e+00,6.186894e+09
4407,77,98.044495,1.5,0.092074,267.174771,1.0,0.418196,237.649627,1.5,0.015146,...,0.000000e+00,0,0.0,0.0,4.771733e+06,3.425450e+04,8.082873e+06,0.000000,0.000000e+00,6.100085e+09
5858,115,207.098402,1.0,0.315454,323.413032,10.0,0.084600,204.869330,10.0,0.050775,...,0.000000e+00,0,0.0,0.0,4.751208e+06,1.350675e+06,7.932134e+06,11443.173949,0.000000e+00,5.993869e+09
3220,106,92.695207,10.0,0.267308,179.748371,1.5,0.659591,309.848294,1.0,0.051963,...,0.000000e+00,0,0.0,0.0,4.872019e+06,0.000000e+00,8.185112e+06,0.000000,0.000000e+00,5.692527e+09
6381,117,169.723118,1.5,0.298968,63.938258,10.0,0.687465,248.828283,1.0,0.077979,...,0.000000e+00,0,0.0,0.0,4.873446e+06,0.000000e+00,7.926118e+06,23728.330854,0.000000e+00,5.687616e+09
5246,44,89.557890,1.0,0.289095,270.471943,1.5,0.916772,53.569762,10.0,0.025754,...,0.000000e+00,0,0.0,0.0,4.817928e+06,0.000000e+00,8.301208e+06,0.000000,0.000000e+00,5.683404e+09
7761,69,209.612261,1.5,0.348515,340.447661,1.5,0.027082,194.752218,1.5,0.031996,...,0.000000e+00,0,0.0,0.0,3.861678e+06,1.941281e+06,7.767134e+06,0.000000,0.000000e+00,5.633348e+09
8098,74,167.762596,1.0,0.307648,336.742864,10.0,0.038736,227.102691,1.5,0.100092,...,0.000000e+00,0,0.0,0.0,4.897874e+06,1.846443e+06,7.694322e+06,0.000000,0.000000e+00,5.553505e+09
4494,23,100.196406,1.0,0.266927,178.828152,1.0,0.077813,279.849696,1.0,0.145576,...,8.744064e+07,0,0.0,0.0,4.868309e+06,1.027346e+06,3.844651e+06,0.000000,1.372036e+05,5.543567e+09


In [33]:
scenarios = []
for row_number in range(results.shape[0]):
    scenarios.append(
        Scenario(name=row_number, **results.iloc[row_number, :16].to_dict())
    )

scenarios[1]

Scenario({'0_RfR 0': 0.0, '0_RfR 1': 0.0, '0_RfR 2': 0.0, '1_RfR 0': 0.0, '1_RfR 1': 0.0, '1_RfR 2': 0.0, '2_RfR 0': 1.0, '2_RfR 1': 1.0, '2_RfR 2': 1.0, '3_RfR 0': 1.0, '3_RfR 1': 0.0, '3_RfR 2': 1.0, '4_RfR 0': 0.0, '4_RfR 1': 1.0, '4_RfR 2': 1.0, 'EWS_DaysToThreat': 2.0})

In [28]:
# List the names of vars to make `robustness_functions` a bit more read-able
var_list_damage = ['A.1_Expected Annual Damage',
                   'A.2_Expected Annual Damage',
                   'A.3_Expected Annual Damage',
                   'A.4_Expected Annual Damage',
                   'A.5_Expected Annual Damage',]
var_list_deaths = ['A.1_Expected Number of Deaths',
                   'A.2_Expected Number of Deaths',
                   'A.3_Expected Number of Deaths',
                   'A.4_Expected Number of Deaths',
                   'A.5_Expected Number of Deaths',]
var_list_dike = ['A.1_Dike Investment Costs',
                 'A.2_Dike Investment Costs',
                 'A.3_Dike Investment Costs',
                 'A.4_Dike Investment Costs',
                 'A.5_Dike Investment Costs',]
var_list_rfr = ['RfR Total Costs',]
var_list_evac = ['Expected Evacuation Costs',]

In [29]:
def sum_sum_sum(*data):
    sumsumsum = sum(sum(sum(data)))

    mean = np.mean(sumsumsum)
    iqr = sp.stats.iqr(sumsumsum) + mean * 0.005
    result = mean + iqr

    return result

# robustness metrics
maximize = ScalarOutcome.MAXIMIZE
minimize = ScalarOutcome.MINIMIZE

# These functions need to only return one score value...
robustness_functions = [
    ScalarOutcome('Damage Score', variable_name=var_list_damage,
                  function=sum_sum_sum, kind=minimize, expected_range=(0, 4e16)),
    ScalarOutcome('Deaths Score', variable_name=var_list_deaths,
                  function=sum_sum_sum, kind=minimize, expected_range=(0, 8.5e19)),
    ScalarOutcome('Dike Invest Score', variable_name=var_list_dike,
                  function=sum_sum_sum, kind=minimize, expected_range=(1e18, 1.3e7)),
    ScalarOutcome("RfR Invest Score", kind=minimize, variable_name="RfR Total Costs", function=sum_sum_sum),
    ScalarOutcome("Evac Score", kind=minimize, variable_name="Expected Evacuation Costs", function=sum_sum_sum),
]

# constraints = [
#     Constraint()
# ]

In [30]:
from ema_workbench.em_framework.optimization import EpsilonProgress
from ema_workbench.em_framework.evaluators import BaseEvaluator

# general input
n_scenarios = 10
#scenarios = sample_uncertainties(dike_model, n_scenarios)
nfe = 200 #make this large

BaseEvaluator.reporting_frequency = 0.1

epsilons = [0.05, ]*len(robustness_functions)

# Run MORO
with MultiprocessingEvaluator(dike_model) as evaluator:
    results, convergence = evaluator.robust_optimize(robustness_functions,
                                                     scenarios=scenarios,
                                                     nfe=nfe,
                                                     epsilons=epsilons,
                                                     convergence=[
                                                         EpsilonProgress()],
                                                     population_size=5,
                                                     )

# Write the results so this step can be skipped when doing multiple analyzes
with open(r'generated_datasets/initial_Pareto_policies_test.pkl', 'wb') as file_pi:
    pickle.dump(results, file_pi)

[MainProcess/INFO] pool started with 12 workers
100%|████████████████████████████████████████| 200/200 [07:18<00:00,  2.19s/it]
[MainProcess/INFO] optimization completed, found 36 solutions
[MainProcess/INFO] terminating pool


In [31]:
results

,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,...,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,Damage Score,Deaths Score,Dike Invest Score,RfR Invest Score,Evac Score
0,0,0,0,0,1,1,0,1,1,1,...,9,4,8,7,8,3.289723e+08,0.019151,1.282910e+10,2.759328e+10,8492.785575
1,0,0,0,0,0,0,1,1,1,1,...,4,4,3,8,6,5.256698e+08,0.030398,1.665022e+10,1.701867e+10,13435.983837
2,0,0,0,1,1,0,1,0,0,0,...,8,5,3,0,4,3.393533e+10,25.453450,9.783276e+09,1.966785e+10,0.000000
3,0,0,0,0,0,0,1,1,1,1,...,4,4,4,3,1,4.491303e+08,0.022343,1.202956e+10,1.701867e+10,13868.855066
4,0,0,0,1,0,0,1,0,1,0,...,7,5,3,7,4,2.322056e+09,0.133354,1.126489e+10,1.590714e+10,67795.551981
5,0,0,0,0,0,0,1,1,1,1,...,4,4,4,3,1,4.491303e+08,0.186191,1.202956e+10,1.701867e+10,0.000000
6,0,0,0,0,0,0,1,1,0,1,...,5,4,4,0,1,4.040926e+10,1.525338,7.930595e+09,1.640160e+10,248069.569703
7,1,0,0,0,0,0,1,1,1,1,...,7,4,4,3,1,4.419574e+08,0.184515,1.114355e+10,1.871913e+10,0.000000
8,1,0,0,0,1,0,1,0,1,1,...,5,5,4,3,1,5.670860e+08,0.043630,1.196181e+10,2.247984e+10,13092.620220
9,1,0,0,0,0,0,0,0,1,1,...,5,6,3,3,2,3.002689e+09,0.135888,1.032857e+10,9.901260e+09,53655.764033


In [ ]:
# Extract the levers for each Pareto policy
policies = []
for row in range(results.shape[0]):
    policies.append(
        # Do not include the damage scores
        Policy(name=row, **results.iloc[row, :-5].to_dict())
    )

with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=50, policies=policies)

# Write the results so this step can be skipped when doing multiple analyzes
with open('Outcomes/epsilon_results.pkl', 'wb') as file_pi:
    pickle.dump(results, file_pi)